In [ ]:
# Installing simpletransformers here 
# because of conflicting transformer version needed by GoEmotions
!pip install simpletransformers

In [ ]:
# Activates CUDA for GPU use for fine-tuning T5 models
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
# Allows user input for:
# 1. Data set to train model
# 2. Training type determines if the training process uses 
# a) The larger Training set
# b) The smaller Test set
# 3. Number of fine-tuning Epochs 
model_name = input('Dataset to train model (mix, twit0.825 or combined):  ')
training_type = input('Regular or limited training (regular or limited): ')
epochs = int(input('Number of training epochs: '))

if training_type.lower() == 'regular':
    train_set = 'training'
    eval_set = 'testing'
elif training_type.lower() == 'limited':
    train_set = 'testing'
    eval_set = 'training'
else:
    print('Please enter a valid training type')

In [ ]:
# Fine-tuning T5 by the specified data set, training type, and number of epochs specified above
import pandas as pd
from simpletransformers.t5 import T5Model

training_df = pd.read_csv(f'emotion-labeled-data/{model_name}/{model_name}-{train_set}_t5.tsv', sep="\t").astype(str)
testing_df = pd.read_csv(f'emotion-labeled-data/{model_name}/{model_name}-{eval_set}_t5.tsv', sep="\t").astype(str)


# General parameters for fine-tuning
model_args = {
    "max_seq_length": 196,
    "train_batch_size": 6,
    "eval_batch_size": 32,
    "num_train_epochs": epochs,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    
    "use_multiprocessing": False,
    "fp16": False,

    "save_steps": -1,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,

    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    
    # Creates special directory to hold each new model
    "output_dir": f"{training_type}-{model_name}-{epochs}epochs", 

}


# Use t5-base as the model to be fine-tuned
model = T5Model("t5", "t5-base", args=model_args)

# Fine-tuning the T5 model
model.train_model(training_df, eval_data=testing_df)

In [ ]:
# Fine-tuning BART by the specified data set, training type, and number of epochs specified above
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

training_df = pd.read_csv(f'emotion-labeled-data/{model_name}/{model_name}-{train_set}_t5.tsv', sep="\t").astype(str)
testing_df = pd.read_csv(f'emotion-labeled-data/{model_name}/{model_name}-{eval_set}_t5.tsv', sep="\t").astype(str)


# General parameters for fine-tuning
model_args = {
    "max_seq_length": 196,
    "train_batch_size": 4,
    "eval_batch_size": 32,
    "num_train_epochs": epochs,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    
    "use_multiprocessing": False,
    "fp16": False,

    "save_steps": -1,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,

    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    
    # Creates special directory to hold each new model
    "output_dir": f"{training_type}-{model_name}-{epochs}epochs-bart", 

}


# Use bart-large as the model to be fine-tuned
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-base",
    args=model_args,
)

# Fine-tuning the bart model
model.train_model(training_df, eval_data=testing_df)